In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Chuy,UY,2022-01-30 22:04:45,-33.6971,-53.4616,73.56,68,20,14.29
1,1,Lebork,PL,2022-01-30 22:04:45,54.5392,17.7501,36.99,67,96,18.23
2,2,Bluff,NZ,2022-01-30 22:02:50,-46.6000,168.3333,68.81,85,93,4.43
3,3,Jamestown,US,2022-01-30 22:01:09,42.0970,-79.2353,20.08,45,100,3.00
4,4,Fortuna,US,2022-01-30 22:00:55,40.5982,-124.1573,60.42,74,0,1.99


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
7,7,Tual,ID,2022-01-30 22:04:46,-5.6667,132.7500,83.73,78,100,22.64
12,12,Noumea,NC,2022-01-30 22:04:47,-22.2763,166.4572,82.99,99,100,9.55
13,13,Tabou,CI,2022-01-30 22:04:47,4.4230,-7.3528,79.79,86,25,7.81
20,20,Angoche,MZ,2022-01-30 22:04:48,-16.2325,39.9086,77.16,82,2,1.77
25,25,Touros,BR,2022-01-30 22:04:49,-5.1989,-35.4608,81.50,76,20,12.57
29,29,Acapulco,MX,2022-01-30 22:04:50,16.8634,-99.8901,84.02,70,20,4.61
30,30,Kapaa,US,2022-01-30 22:04:50,22.0752,-159.3190,80.47,72,0,1.01
31,31,Nuqui,CO,2022-01-30 22:04:50,5.7125,-77.2708,78.40,84,80,5.88
32,32,Cabo San Lucas,MX,2022-01-30 22:04:51,22.8909,-109.9124,79.61,60,20,1.99
39,39,Kahului,US,2022-01-30 22:01:01,20.8947,-156.4700,78.84,68,4,15.99


In [23]:
preferred_cities_df.count()

City_ID       131
City          131
Country       131
Date          131
Lat           131
Lng           131
Max Temp      131
Humidity      131
Cloudiness    131
Wind Speed    131
dtype: int64

In [24]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Tual,ID,83.73,-5.6667,132.7500,
12,Noumea,NC,82.99,-22.2763,166.4572,
13,Tabou,CI,79.79,4.4230,-7.3528,
20,Angoche,MZ,77.16,-16.2325,39.9086,
25,Touros,BR,81.50,-5.1989,-35.4608,
29,Acapulco,MX,84.02,16.8634,-99.8901,
30,Kapaa,US,80.47,22.0752,-159.3190,
31,Nuqui,CO,78.40,5.7125,-77.2708,
32,Cabo San Lucas,MX,79.61,22.8909,-109.9124,
39,Kahului,US,78.84,20.8947,-156.4700,


In [25]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [29]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [35]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [36]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))